### Data Ingestion

In [ ]:
from langchain_core.documents import Document


In [ ]:
# Single file load
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../data/text_files/eg.txt', encoding='utf8')
document= loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/eg.txt'}, page_content='Hello, my name is Shree Mengshetti. I am applying for the AI Research Engineer Internship role. Please find my resume attached for your kind consideration.')]


In [ ]:
# Multiple file load
from langchain_community.document_loaders import DirectoryLoader
